In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

In [27]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
path = '/kaggle/input/playground-series-s5e8/'

if not iskaggle:
    path = f'{os.getcwd()}/playground-series-s5e8'

train_data = pd.read_csv(f'{path}/train.csv')
test_data = pd.read_csv(f'{path}/test.csv')

In [28]:
categorical_cols = [col for col in train_data.columns if train_data[col].dtype == "object"]

for feature in categorical_cols:
        train_data[feature] = train_data[feature].astype("category")
for feature in categorical_cols:
        test_data[feature] = test_data[feature].astype("category")

In [29]:
train_data_features = train_data.drop(columns=['y'])
train_data_target = train_data['y']

# Train LGBMClassifier with default parameters

In [30]:
X_train_default, X_val_default, y_train_default, y_val_default = train_test_split(train_data_features, train_data_target, test_size=0.2, random_state=42)

In [31]:
lgb_clf_default = lgb.LGBMClassifier(verbosity=1)
lgb_clf_default.fit(X_train_default, y_train_default)
y_probs_lgb_default = lgb_clf_default.predict_proba(X_val_default)[:,1]

[LightGBM] [Info] Number of positive: 72283, number of negative: 527717
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1262
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120472 -> initscore=-1.987971
[LightGBM] [Info] Start training from score -1.987971


In [32]:
lgb_clf_ChosenParams = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.06,
    num_leaves=100,
    max_depth=10,
    min_child_samples=9,
    subsample=0.8,
    colsample_bytree=0.5,
    reg_alpha=0.79,
    reg_lambda=3.0,
    max_bin=4523,
    random_state=42,
    verbosity=-1,
    n_jobs=-1,
    class_weight='balanced'  # Added parameter to handle class imbalance
)

lgb_clf_ChosenParams.fit(
    X_train_default, 
    y_train_default, 
    eval_set=[(X_val_default, y_val_default)], 
    callbacks=[
        lgb.early_stopping(100),
        lgb.log_evaluation(period=500)
    ]
)

y_probs_lgb_ChosenParams = lgb_clf_ChosenParams.predict_proba(X_val_default)[:,1]

Training until validation scores don't improve for 100 rounds
[500]	valid_0's binary_logloss: 0.215967
[1000]	valid_0's binary_logloss: 0.201796
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.201796


In [10]:
%%time
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

y_probs_lgb = np.zeros(len(test_data))          # For averaged test set predictions
y_probs_val_lgb = np.zeros(len(train_data_features))           # For out-of-fold validation predictions

for fold, (train_idx, val_idx) in enumerate(kf.split(train_data_features, train_data_target)):
    print(f"Training fold {fold + 1}/{n_splits} >>>")
    X_train, y_train = train_data_features.iloc[train_idx], train_data_target.iloc[train_idx]
    X_val, y_val = train_data_features.iloc[val_idx], train_data_target.iloc[val_idx]

    lgb_clf = lgb.LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.06,
        num_leaves=100,
        max_depth=10,
        min_child_samples=9,
        subsample=0.8,
        colsample_bytree=0.5,
        reg_alpha=0.79,
        reg_lambda=3.0,
        max_bin=4523,
        random_state=42,
        verbosity=-1,
        n_jobs=-1,
        class_weight='balanced'  # Added parameter to handle class imbalance
    )
    
    lgb_clf.fit(
        X_train, 
        y_train, 
        eval_set=[(X_val, y_val)], 
        callbacks=[
            lgb.early_stopping(100),
            lgb.log_evaluation(period=500)
        ]
    )

    # Save validation fold predictions into the correct positions of full OOF array
    y_probs_val_lgb[val_idx] = lgb_clf.predict_proba(X_val)[:, 1]

    # Accumulate test predictions averaged over folds
    y_probs_lgb += lgb_clf.predict_proba(test_data)[:, 1] / n_splits


Training fold 1/5 >>>
Training until validation scores don't improve for 100 rounds
[500]	valid_0's binary_logloss: 0.214654
[1000]	valid_0's binary_logloss: 0.200212
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.200212
Training fold 2/5 >>>
Training until validation scores don't improve for 100 rounds
[500]	valid_0's binary_logloss: 0.215758
[1000]	valid_0's binary_logloss: 0.201244
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.201244
Training fold 3/5 >>>
Training until validation scores don't improve for 100 rounds
[500]	valid_0's binary_logloss: 0.218032
[1000]	valid_0's binary_logloss: 0.204124
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.204124
Training fold 4/5 >>>
Training until validation scores don't improve for 100 rounds
[500]	valid_0's binary_logloss: 0.216303
[1000]	valid_0's binary_logloss: 0.201836
Did not meet early stopping. Best iteration is:
[1000]	valid_0

In [47]:
%%time
folds = 3
param_comb = 20

params = {
    'n_estimators' : [100, 1000, 2000],
    'learning_rate' : [0.01, 0.06, 0.1],
    'num_leaves': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_child_samples' : [5, 9, 15],
    'subsample' : [0.8],
    'colsample_bytree' : [0.5],
    'reg_alpha' : [0.79],
    'reg_lambda' : [3.0],
    'max_bin' : [4523]
}


lgb_clf = lgb.LGBMClassifier(
    random_state=42,
    verbosity=1,
    n_jobs=1,
    class_weight='balanced'
)

# skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=lgb_clf, 
    param_distributions=params, 
    n_iter=param_comb,
    scoring='roc_auc', 
    n_jobs=-1,
#     cv=skf.split(X_train_default, y_train_default), 
    cv=folds,
    verbose=1,
    random_state=42
#     return_train_score=True
)
# random_search = RandomizedSearchCV(
#     estimator=lgbm,
#     param_distributions=param_distributions,
#     n_iter=50,              # Number of parameter combinations to try
#     cv=5,                   # 5-fold cross-validation
#     scoring='accuracy',     # Metric to optimize
#     n_jobs=-1,             # Use all available cores
#     random_state=42,
#     verbose=1              # Show progress
# )

# random_search.fit(train_data_features, train_data_target)
random_search.fit(X_train_default, y_train_default)

# y_probs_val_rand = random_search.predict_proba(train_data_features)[:,1]
y_probs_val_rand = random_search.predict_proba(X_val_default)[:,1]

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Number of positive: 48188, number of negative: 351812
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026531 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10839
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[LightGBM] [Info] Number of positive: 48189, number of negative: 351811
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022474 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10848
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[LightGBM] [Info] Number of positive: 48189, number of negative: 351811
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021158 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10848
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[LightGBM] [Info] Number of positive: 48188, number of negative: 351812
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028449 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10839
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[LightGBM] [Info] Number of positive: 48189, number of negative: 351811
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026683 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10868
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Number of positive: 72283, number of negative: 527717
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025984 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10841
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
CPU times: user 1min 36s, sys: 2.75 s, total: 1min 39s
Wall time: 22min 48s


In [24]:
from random import randint 
randint(50, 500)

289

In [49]:
roc_auc_lgbDefault = roc_auc_score(y_val_default, y_probs_lgb_default)
roc_auc_lgbChosenParams = roc_auc_score(y_val_default, y_probs_lgb_ChosenParams)
# oof_auc_lgb = roc_auc_score(train_data_target, y_probs_val_lgb)
rand_auc_score = roc_auc_score(y_val_default, y_probs_val_rand)

print(f'LightGBM default: {roc_auc_lgbDefault:.5f}')
print(f'LightGBM ChosenParams: {roc_auc_lgbChosenParams:.5f}')
# print(f"LightGBM StratifiedKFold: {oof_auc_lgb:.5f}")
print(f"LightGBM RandomSearchCV: {rand_auc_score:.5f}")

LightGBM default: 0.96504
LightGBM ChosenParams: 0.97209
LightGBM RandomSearchCV: 0.97207


In [21]:
lgb_pred = lgb_clf_ChosenParams.predict_proba(test_data)[:,1]
print(lgb_pred)

lgb_rand_pred = random_search.predict_proba(test_data)[:,1]
print(lgb_pred)

[5.96276398e-03 1.83628128e-01 3.37798947e-04 ... 8.42236664e-01
 1.22440696e-03 2.27536844e-01]
[5.96276398e-03 1.83628128e-01 3.37798947e-04 ... 8.42236664e-01
 1.22440696e-03 2.27536844e-01]


In [22]:
def submit(df, pred, fname):
    submission = pd.DataFrame({'id' : df['id'], 'y':pred})
    submission = submission.sort_values(by="y", ascending=True)
    submission.to_csv(fname, index=False)

submit(test_data, lgb_pred, 'submission_lgb_TrainTestSplit.csv')
submit(test_data, lgb_rand_pred, 'submission_lgb_TrainTestSplit.csv')

In [ ]:
%%time
folds = 3
param_comb = 1
# param_comb = 20

params = {
    'n_estimators' : [1000],
    'learning_rate' : [0.06],
    'num_leaves': [100],
    'max_depth': [10],
    'min_child_samples' : [9],
    'subsample' : [0.8],
    'colsample_bytree' : [0.5],
    'reg_alpha' : [0.79],
    'reg_lambda' : [3.0],
    'max_bin' : [4523]
}

lgb_clf = lgb.LGBMClassifier(
    random_state=42,
    verbosity=1,
    n_jobs=1,
    class_weight='balanced'
)

random_search = RandomizedSearchCV(
    estimator=lgb_clf, 
    param_distributions=params, 
    n_iter=param_comb,
    scoring='roc_auc', 
    n_jobs=-1,
    cv=folds,
    verbose=1,
    random_state=42
)

random_search.fit(X_train_default, y_train_default)

y_probs_val_rand = random_search.predict_proba(X_val_default)[:,1]